In [128]:
import pandas as pd
from tqdm import tqdm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


new = pd.DataFrame()
df_predictions = pd.DataFrame

df_accuracy = pd.read_csv('top_and_bottom_anime.csv')
df_accuracy = df_accuracy.drop(columns='Id')
df_accuracy = df_accuracy.drop(columns='Rating')


le = preprocessing.LabelEncoder()
for i in range(df_accuracy.shape[1]):
    temp = df_accuracy.iloc[:,i].values.tolist()
    temp = le.fit_transform(temp)
    new[df_accuracy.columns[i]] = temp

df_true = pd.read_csv('anime_matrix_true.csv')
df_predictions = df_true[['Id']].copy()
df_true = df_true.drop(columns='Id')

columns = df_true.columns
predictions = []
x = new
y = df_true[columns[1]]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=42)
for i in tqdm(range(len(columns))):
    y = df_true[columns[i]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=42)
    adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    adaboost.fit(x_train,y_train)
    result = adaboost.score(x_test,y_test)
    predictions.append(list(y_test))

result


100%|██████████| 9028/9028 [13:09<00:00, 11.43it/s]


0.8660714285714286

In [133]:
import warnings

cut_off = (len(df_predictions)) - (len(y_test))
df_predictions.drop(df_predictions.tail(cut_off).index,inplace=True)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
df_predictions = pd.DataFrame()
for i in tqdm(range(len(columns))):
    df_predictions[columns[i]] = predictions[i]

100%|██████████| 9028/9028 [00:04<00:00, 2022.38it/s]


In [130]:
import pickle

# open a file, where you ant to store the data
file = open('adaboost.model', 'wb')

# dump information to that file
pickle.dump(adaboost, file)

# close the file
file.close()

In [131]:
outputdf = df_predictions.copy()
outputdf.to_csv(f'predictions_matrix_adaboost.csv', index=False)